# Capstone Project: Data Collection     

29/11/2020

We collect restaurant data via the Foursquare API.

In [1]:
import pandas as pd
import numpy as np
import requests 
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install folium
import folium # map rendering library


In [2]:
# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# create map of Toronto using latitude and longitude values
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=11)

map_warsaw

In [3]:
# @hidden_cell
CLIENT_ID = '34CRGBIL11CRLO5BNFSRKHWNRQT1GWTB4QKGYG5GLQYM3JKQ' # your Foursquare ID
CLIENT_SECRET = 'HRPY3HMJC1FSPYTC2VRVUANCY1JY20NBRA5400RQYOEOFCNB' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
VERSION = '20201123' # Foursquare API version
RADIUS=500
LIMIT = 50 # A default Foursquare API limit value

In [4]:
# List of categories
afghan = '503288ae91d4c4b30a586d67'
african = '4bf58dd8d48988d1c8941735'
american = '4bf58dd8d48988d14e941735'
asian = '4bf58dd8d48988d142941735'
australian = '4bf58dd8d48988d169941735'
austrian = '52e81612bcbc57f1066b7a01'
bangladesh = '5e179ee74ae8e90006e9a746'
belgium = '52e81612bcbc57f1066b7a02'
cajun = '4bf58dd8d48988d17a941735'
caribbean = '4bf58dd8d48988d144941735'
caucasian = '5293a7d53cf9994f4e043a45'
czech = '52f2ae52bcbc57f1066b8b81'
dutch = '5744ccdfe4b0c0459246b4d0'
eastEurope = '4bf58dd8d48988d109941735'
english = '52e81612bcbc57f1066b7a05'
french = '4bf58dd8d48988d10c941735'
german = '4bf58dd8d48988d10d941735'
greek = '4bf58dd8d48988d10e941735'
hungary = '52e81612bcbc57f1066b79fa'
indian = '4bf58dd8d48988d10f941735'
italian = '4bf58dd8d48988d110941735'
jewish = '52e81612bcbc57f1066b79fd'
latin = '4bf58dd8d48988d1be941735'
meditarranean = '4bf58dd8d48988d1c0941735'
mexican = '4bf58dd8d48988d1c1941735'
middleEast = '4bf58dd8d48988d115941735'
modernEuropean = '52e81612bcbc57f1066b79f9'
pakistan = '52e81612bcbc57f1066b79f8'
polish = '52e81612bcbc57f1066b7a04'
portugal = '4def73e84765ae376e57713a'
russia = '5293a7563cf9994f4e043a44'
scandinavia = '4bf58dd8d48988d1c6941735'
scotland = '5744ccdde4b0c0459246b4a3'
slovak = '56aa371be4b08b9a8d57355a'
spain = '4bf58dd8d48988d150941735'
srilanka = '5413605de4b0ae91d18581a9'
swiss = '4bf58dd8d48988d158941735'
turkey = '4f04af1f2fb6e1c99f3db0bb'
ukraine = '52e928d0bcbc57f1066b7e96'

category_list = [afghan,african,american,asian,australian,austrian,bangladesh,belgium,cajun,caribbean,caucasian,czech,dutch,eastEurope,english,
                 french,german,greek,hungary,indian,italian,jewish,latin,meditarranean,mexican,middleEast,modernEuropean,pakistan,polish,
                 portugal,russia,scandinavia,scotland,slovak,spain,srilanka,swiss,turkey,ukraine] 

CATEGORIES = ','.join(category_list)

In [5]:
def getVenues(latitude, longitude, RADIUS=500):
    
    url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse'.format(
            CATEGORIES,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS,
            LIMIT)
    
    response = requests.get(url)
    results = response.json()
    
    if response.status_code == 200:
        venues = results['response']['venues']
    else:
        venues = []
        print("HTTP-{}".format(response.status_code))
        
    venue_list = []
    
    number_of_venues = len(venues)
    
    for i in range(number_of_venues):
        
        try:
            venue_id = venues[i]['id']
            venue_name = venues[i]['name']
            lat = venues[i]['location']['lat']
            lng = venues[i]['location']['lng']
            category = venues[i]['categories'][0]['name']
            
            record = (latitude, longitude, number_of_venues, i, venue_id, venue_name, lat, lng, category)
        except:
            record = (latitude, longitude, number_of_venues, i, '', '', '', '', '')
        
        venue_list.append(record)
        
        
    return( venue_list )

# Warsaw

In [14]:
# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# Box boundaries
y_max = latitude + 0.1
y_min = latitude - 0.1
x_max = longitude + 0.20
x_min = longitude - 0.15


lat_centre = (y_max+y_min)/2.0

y_step = 1.0/(110.574)
x_step = 1.0/(111.32*np.cos(2.0*np.pi*lat_centre/360.0))

num_x_points = int((x_max-x_min)/x_step)
num_y_points = int((y_max-y_min)/y_step)

lats = np.linspace(x_min,x_max,num_x_points)
longs = np.linspace(y_min,y_max,num_y_points)

' Create matrix of coordinate values '
coord_list = []
for x in longs:
    for y in lats:
        
        coord_list.append( (x,y) )

print("Number of points = {}".format(len(coord_list)))

Number of points = 506


In [16]:
# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# create map of Toronto using latitude and longitude values
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_warsaw)  
    
map_warsaw

## Retrieve data from Foursquare

In [17]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('warsaw_20201205_2220.csv',encoding='utf-8')

lat = 52.1297 long = 20.8622 Number of Venues = 0
lat = 52.1297 long = 20.8781090909 Number of Venues = 0
lat = 52.1297 long = 20.8940181818 Number of Venues = 5
lat = 52.1297 long = 20.9099272727 Number of Venues = 0
lat = 52.1297 long = 20.9258363636 Number of Venues = 0
lat = 52.1297 long = 20.9417454545 Number of Venues = 0
lat = 52.1297 long = 20.9576545455 Number of Venues = 0
lat = 52.1297 long = 20.9735636364 Number of Venues = 0
lat = 52.1297 long = 20.9894727273 Number of Venues = 0
lat = 52.1297 long = 21.0053818182 Number of Venues = 1
lat = 52.1297 long = 21.0212909091 Number of Venues = 5
lat = 52.1297 long = 21.0372 Number of Venues = 0
lat = 52.1297 long = 21.0531090909 Number of Venues = 11
lat = 52.1297 long = 21.0690181818 Number of Venues = 21
lat = 52.1297 long = 21.0849272727 Number of Venues = 1
lat = 52.1297 long = 21.1008363636 Number of Venues = 3
lat = 52.1297 long = 21.1167454545 Number of Venues = 0
lat = 52.1297 long = 21.1326545455 Number of Venues = 0
la

lat = 52.1773190476 long = 21.2122 Number of Venues = 0
lat = 52.1868428571 long = 20.8622 Number of Venues = 0
lat = 52.1868428571 long = 20.8781090909 Number of Venues = 1
lat = 52.1868428571 long = 20.8940181818 Number of Venues = 13
lat = 52.1868428571 long = 20.9099272727 Number of Venues = 5
lat = 52.1868428571 long = 20.9258363636 Number of Venues = 0
lat = 52.1868428571 long = 20.9417454545 Number of Venues = 3
lat = 52.1868428571 long = 20.9576545455 Number of Venues = 9
lat = 52.1868428571 long = 20.9735636364 Number of Venues = 1
lat = 52.1868428571 long = 20.9894727273 Number of Venues = 12
lat = 52.1868428571 long = 21.0053818182 Number of Venues = 12
lat = 52.1868428571 long = 21.0212909091 Number of Venues = 16
lat = 52.1868428571 long = 21.0372 Number of Venues = 1
lat = 52.1868428571 long = 21.0531090909 Number of Venues = 9
lat = 52.1868428571 long = 21.0690181818 Number of Venues = 8
lat = 52.1868428571 long = 21.0849272727 Number of Venues = 0
lat = 52.1868428571 lo

lat = 52.2344619048 long = 21.1485636364 Number of Venues = 1
lat = 52.2344619048 long = 21.1644727273 Number of Venues = 0
lat = 52.2344619048 long = 21.1803818182 Number of Venues = 0
lat = 52.2344619048 long = 21.1962909091 Number of Venues = 0
lat = 52.2344619048 long = 21.2122 Number of Venues = 0
lat = 52.2439857143 long = 20.8622 Number of Venues = 0
lat = 52.2439857143 long = 20.8781090909 Number of Venues = 0
lat = 52.2439857143 long = 20.8940181818 Number of Venues = 1
lat = 52.2439857143 long = 20.9099272727 Number of Venues = 5
lat = 52.2439857143 long = 20.9258363636 Number of Venues = 0
lat = 52.2439857143 long = 20.9417454545 Number of Venues = 13
lat = 52.2439857143 long = 20.9576545455 Number of Venues = 10
lat = 52.2439857143 long = 20.9735636364 Number of Venues = 3
lat = 52.2439857143 long = 20.9894727273 Number of Venues = 50
lat = 52.2439857143 long = 21.0053818182 Number of Venues = 39
lat = 52.2439857143 long = 21.0212909091 Number of Venues = 30
lat = 52.243985

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/search?&categoryId=503288ae91d4c4b30a586d67,4bf58dd8d48988d1c8941735,4bf58dd8d48988d14e941735,4bf58dd8d48988d142941735,4bf58dd8d48988d169941735,52e81612bcbc57f1066b7a01,5e179ee74ae8e90006e9a746,52e81612bcbc57f1066b7a02,4bf58dd8d48988d17a941735,4bf58dd8d48988d144941735,5293a7d53cf9994f4e043a45,52f2ae52bcbc57f1066b8b81,5744ccdfe4b0c0459246b4d0,4bf58dd8d48988d109941735,52e81612bcbc57f1066b7a05,4bf58dd8d48988d10c941735,4bf58dd8d48988d10d941735,4bf58dd8d48988d10e941735,52e81612bcbc57f1066b79fa,4bf58dd8d48988d10f941735,4bf58dd8d48988d110941735,52e81612bcbc57f1066b79fd,4bf58dd8d48988d1be941735,4bf58dd8d48988d1c0941735,4bf58dd8d48988d1c1941735,4bf58dd8d48988d115941735,52e81612bcbc57f1066b79f9,52e81612bcbc57f1066b79f8,52e81612bcbc57f1066b7a04,4def73e84765ae376e57713a,5293a7563cf9994f4e043a44,4bf58dd8d48988d1c6941735,5744ccdde4b0c0459246b4a3,56aa371be4b08b9a8d57355a,4bf58dd8d48988

In [27]:
i=0
for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    print lat,lng,i
    
    i = i + 1

52.1297 20.8622 0
52.1297 20.878109090909092 1
52.1297 20.894018181818183 2
52.1297 20.909927272727273 3
52.1297 20.925836363636364 4
52.1297 20.941745454545455 5
52.1297 20.957654545454545 6
52.1297 20.973563636363636 7
52.1297 20.989472727272727 8
52.1297 21.005381818181817 9
52.1297 21.021290909090908 10
52.1297 21.037200000000002 11
52.1297 21.053109090909093 12
52.1297 21.069018181818183 13
52.1297 21.084927272727274 14
52.1297 21.100836363636365 15
52.1297 21.116745454545455 16
52.1297 21.132654545454546 17
52.1297 21.148563636363637 18
52.1297 21.164472727272727 19
52.1297 21.180381818181818 20
52.1297 21.19629090909091 21
52.1297 21.2122 22
52.13922380952381 20.8622 23
52.13922380952381 20.878109090909092 24
52.13922380952381 20.894018181818183 25
52.13922380952381 20.909927272727273 26
52.13922380952381 20.925836363636364 27
52.13922380952381 20.941745454545455 28
52.13922380952381 20.957654545454545 29
52.13922380952381 20.973563636363636 30
52.13922380952381 20.9894727272727

In [29]:
new_coord_list = coord_list[348:]

In [31]:
record_list = []

for coord in new_coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('warsaw_20201205_2340.csv',encoding='utf-8')

lat = 52.2725571429 long = 20.9099272727 Number of Venues = 0
lat = 52.2725571429 long = 20.9258363636 Number of Venues = 5
lat = 52.2725571429 long = 20.9417454545 Number of Venues = 5
lat = 52.2725571429 long = 20.9576545455 Number of Venues = 4
lat = 52.2725571429 long = 20.9735636364 Number of Venues = 7
lat = 52.2725571429 long = 20.9894727273 Number of Venues = 6
lat = 52.2725571429 long = 21.0053818182 Number of Venues = 0
lat = 52.2725571429 long = 21.0212909091 Number of Venues = 1
lat = 52.2725571429 long = 21.0372 Number of Venues = 1
lat = 52.2725571429 long = 21.0531090909 Number of Venues = 4
lat = 52.2725571429 long = 21.0690181818 Number of Venues = 0
lat = 52.2725571429 long = 21.0849272727 Number of Venues = 1
lat = 52.2725571429 long = 21.1008363636 Number of Venues = 1
lat = 52.2725571429 long = 21.1167454545 Number of Venues = 1
lat = 52.2725571429 long = 21.1326545455 Number of Venues = 0
lat = 52.2725571429 long = 21.1485636364 Number of Venues = 0
lat = 52.27255

lat = 52.3297 long = 20.8622 Number of Venues = 0
lat = 52.3297 long = 20.8781090909 Number of Venues = 1
lat = 52.3297 long = 20.8940181818 Number of Venues = 1
lat = 52.3297 long = 20.9099272727 Number of Venues = 1
lat = 52.3297 long = 20.9258363636 Number of Venues = 0
lat = 52.3297 long = 20.9417454545 Number of Venues = 3
lat = 52.3297 long = 20.9576545455 Number of Venues = 1
lat = 52.3297 long = 20.9735636364 Number of Venues = 1
lat = 52.3297 long = 20.9894727273 Number of Venues = 0
lat = 52.3297 long = 21.0053818182 Number of Venues = 0
lat = 52.3297 long = 21.0212909091 Number of Venues = 0
lat = 52.3297 long = 21.0372 Number of Venues = 0
lat = 52.3297 long = 21.0531090909 Number of Venues = 0
lat = 52.3297 long = 21.0690181818 Number of Venues = 0
lat = 52.3297 long = 21.0849272727 Number of Venues = 0
lat = 52.3297 long = 21.1008363636 Number of Venues = 0
lat = 52.3297 long = 21.1167454545 Number of Venues = 1
lat = 52.3297 long = 21.1326545455 Number of Venues = 1
lat 

In [32]:
df_venues

lat_centre  long_centre  num_venues  venue_iter  \
0     52.272557    20.925836           5           0   
1     52.272557    20.925836           5           1   
2     52.272557    20.925836           5           2   
3     52.272557    20.925836           5           3   
4     52.272557    20.925836           5           4   
5     52.272557    20.941745           5           0   
6     52.272557    20.941745           5           1   
7     52.272557    20.941745           5           2   
8     52.272557    20.941745           5           3   
9     52.272557    20.941745           5           4   
10    52.272557    20.957655           4           0   
11    52.272557    20.957655           4           1   
12    52.272557    20.957655           4           2   
13    52.272557    20.957655           4           3   
14    52.272557    20.973564           7           0   
15    52.272557    20.973564           7           1   
16    52.272557    20.973564           7           2   
17    52.272557    20.973564           7           3   
18    52.272557    20.973564           7           4   
19    52.272557    20.973564           7           5   
20    52.272557    20.973564           7           6   
21    52.272557    20.989473           6           0   
22    52.272557    20.989473           6           1   
23    52.272557    20.989473           6           2   
24    52.272557    20.989473           6           3   
25    52.272557    20.989473           6           4   
26    52.272557    20.989473           6           5   
27    52.272557    21.021291           1           0   
28    52.272557    21.037200           1           0   
29    52.272557    21.053109           4           0   
..          ...          ...         ...         ...   
139   52.320176    20.941745           9           0   
140   52.320176    20.941745           9           1   
141   52.320176    20.941745           9           2   
142   52.320176    20.941745           9           3   
143   52.320176    20.941745           9           4   
144   52.320176    20.941745           9           5   
145   52.320176    20.941745           9           6   
146   52.320176    20.941745           9           7   
147   52.320176    20.941745           9           8   
148   52.320176    20.957655           3           0   
149   52.320176    20.957655           3           1   
150   52.320176    20.957655           3           2   
151   52.320176    21.005382           4           0   
152   52.320176    21.005382           4           1   
153   52.320176    21.005382           4           2   
154   52.320176    21.005382           4           3   
155   52.320176    21.053109           2           0   
156   52.320176    21.053109           2           1   
157   52.320176    21.069018           1           0   
158   52.320176    21.084927           1           0   
159   52.329700    20.878109           1           0   
160   52.329700    20.894018           1           0   
161   52.329700    20.909927           1           0   
162   52.329700    20.941745           3           0   
163   52.329700    20.941745           3           1   
164   52.329700    20.941745           3           2   
165   52.329700    20.957655           1           0   
166   52.329700    20.973564           1           0   
167   52.329700    21.116745           1           0   
168   52.329700    21.132655           1           0   

                     venue_id                               venue_name  \
0    52e7d84b498e061d62ab1e5f                 Lotos Kuchnia Orientalna   
1    57f90cd3498eacc89d7bb63b                               Curry Leaf   
2    597b33431499465b3af665f2                                 Pretekst   
3    5a7de7eb6bd36b28336986e7                                  Bao Bao   
4    5bf7db5da6031c002cd8cd28                       chata u kowalskich   
5    56cc90cecd104ce48f009a7a                             Zakątek Azji   
6    4ebdc95cc

In [33]:
import os
files = os.listdir('.')

file_pattern = 'warsaw_'

venue_files = []

' Obtain all files holding venue info '
for filename in files:
    if file_pattern in filename:
        venue_files.append(filename)

' Read in files into dataframe '
df_venues = pd.DataFrame()
for filename in venue_files:
    df = pd.read_csv(filename,encoding='utf-8')
    print( "File: {} Records: {}".format(filename,len(df)) )
    df_venues = df_venues.append(df, ignore_index = True) 

File: warsaw_20201205_2220.csv Records: 1712
File: warsaw_20201205_2340.csv Records: 169


In [35]:
df_venues.head()

Unnamed: 0  lat_centre  long_centre  num_venues  venue_iter  \
0           0     52.1297    20.894018           5           0   
1           1     52.1297    20.894018           5           1   
2           2     52.1297    20.894018           5           2   
3           3     52.1297    20.894018           5           3   
4           4     52.1297    20.894018           5           4   

                   venue_id                venue_name  lat_venue  long_venue  \
0  4d62a5559f67f04d6d7d91fb          Restauracja IKEA  52.133873   20.895375   
1  524988de2fc66ad5ee3dea1d                   kołacze  52.132711   20.894022   
2  57fb79c2498e83e37a64e85e  Buddha Indian Restaurant  52.127865   20.895880   
3  4de6764b45dda52a060ddeb2                    Nikola  52.132072   20.897051   
4  5224698811d26386192fd754              Bar Sajgonki  52.133190   20.897552   

                      category  
0      Scandinavian Restaurant  
1  Eastern European Restaurant  
2      South Indian Restaurant  
3            Indian Restaurant  
4             Asian Restaurant

In [41]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 6 sites


array([[52.2249381 , 21.00538182],
       [52.2249381 , 21.02129091],
       [52.2344619 , 20.98947273],
       [52.2344619 , 21.00538182],
       [52.2344619 , 21.02129091],
       [52.24398571, 20.98947273]])

In [42]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()


In [43]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/4.0)
    xright = x + (x_step/4.0)
    ytop = y + (y_step/4.0)
    ybottom = y - (y_step/4.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [44]:
# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# create map of Toronto using latitude and longitude values
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_warsaw)  
    
map_warsaw

## Retrieve info from Foursquare

In [45]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=250)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('warsaw_20201206_0005.csv',encoding='utf-8')

lat = 52.2249380952 long = 21.0053818182 Number of Venues = 19
lat = 52.2271990246 long = 21.0017156195 Number of Venues = 18
lat = 52.2226771659 long = 21.0017156195 Number of Venues = 3
lat = 52.2271990246 long = 21.0090480168 Number of Venues = 35
lat = 52.2226771659 long = 21.0090480168 Number of Venues = 50
lat = 52.2249380952 long = 21.0212909091 Number of Venues = 35
lat = 52.2271990246 long = 21.0176247104 Number of Venues = 43
lat = 52.2226771659 long = 21.0176247104 Number of Venues = 44
lat = 52.2271990246 long = 21.0249571077 Number of Venues = 18
lat = 52.2226771659 long = 21.0249571077 Number of Venues = 3
lat = 52.2344619048 long = 20.9894727273 Number of Venues = 18
lat = 52.2367228341 long = 20.9858057421 Number of Venues = 19
lat = 52.2322009754 long = 20.9858057421 Number of Venues = 16
lat = 52.2367228341 long = 20.9931397125 Number of Venues = 23
lat = 52.2322009754 long = 20.9931397125 Number of Venues = 20
lat = 52.2344619048 long = 21.0053818182 Number of Venues

In [46]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 3 sites


array([[52.22267717, 21.00904802],
       [52.23220098, 21.01762392],
       [52.2344619 , 21.02129091]])

## Calculate new site locations at breached points

In [47]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()


In [48]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/8.0)
    xright = x + (x_step/8.0)
    ytop = y + (y_step/8.0)
    ybottom = y - (y_step/8.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [49]:
# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# create map of Toronto using latitude and longitude values
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_warsaw)  
    
map_warsaw

## Retrieve info from Foursquare

In [50]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=100)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('warsaw_20201206_0010.csv',encoding='utf-8')

lat = 52.2226771659 long = 21.0090480168 Number of Venues = 8
lat = 52.2238076306 long = 21.0072150108 Number of Venues = 14
lat = 52.2215467012 long = 21.0072150108 Number of Venues = 0
lat = 52.2238076306 long = 21.0108810228 Number of Venues = 10
lat = 52.2215467012 long = 21.0108810228 Number of Venues = 10
lat = 52.2322009754 long = 21.0176239239 Number of Venues = 10
lat = 52.2333314401 long = 21.0157905246 Number of Venues = 15
lat = 52.2310705108 long = 21.0157905246 Number of Venues = 8
lat = 52.2333314401 long = 21.0194573231 Number of Venues = 36
lat = 52.2310705108 long = 21.0194573231 Number of Venues = 7
lat = 52.2344619048 long = 21.0212909091 Number of Venues = 9
lat = 52.2355923694 long = 21.0194574165 Number of Venues = 11
lat = 52.2333314401 long = 21.0194574165 Number of Venues = 36
lat = 52.2355923694 long = 21.0231244017 Number of Venues = 1
lat = 52.2333314401 long = 21.0231244017 Number of Venues = 1


## Combine all files into single file

In [51]:
import os
files = os.listdir('.')

file_pattern = 'warsaw_'

venue_files = []

' Obtain all files holding venue info '
for filename in files:
    if file_pattern in filename:
        venue_files.append(filename)

' Read in files into dataframe '
df_venues = pd.DataFrame()
for filename in venue_files:
    df = pd.read_csv(filename,encoding='utf-8')
    print( "File: {} Records: {}".format(filename,len(df)) )
    df_venues = df_venues.append(df, ignore_index = True) 

File: warsaw_20201206_0010.csv Records: 176
File: warsaw_20201206_0005.csv Records: 723
File: warsaw_20201205_2220.csv Records: 1712
File: warsaw_20201205_2340.csv Records: 169


## Remove duplicate entries and clean up data

In [52]:
df_venues.sort_values("venue_id", inplace = True)
df_venues.drop_duplicates(subset ="venue_id", keep = False, inplace = True)
df_venues.to_csv('warsaw_all.csv',encoding='utf-8')

In [53]:
' Keep only venues with "Restaurant" in their category title '
df_venues['restaurant'] = df_venues['category'].apply(lambda x: True if 'Restaurant' in x else False)
df_venues = df_venues[df_venues['restaurant'] == True]
df_venues.drop('restaurant', axis=1, inplace=True)
df_venues.to_csv('warsaw_restaurants.csv',encoding='utf-8')

In [55]:
from folium.plugins import FastMarkerCluster

# Coordinates of Warsaw
latitude = 52.2297
longitude = 21.0122

# create map of Zurich using latitude and longitude values
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=10)

FastMarkerCluster(data=list(zip(df_venues['lat_venue'],df_venues['long_venue']))).add_to(map_warsaw)

folium.LayerControl().add_to(map_warsaw);

map_warsaw